# Smart Beta



In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
spy = pd.read_excel('./data/spy_components.xlsx')
ticker = spy['Ticker'].dropna().tolist()
ticker_string = ' '.join(ticker)
ticker_string = ticker_string.replace('.', '-')
tickers = yf.Tickers(ticker_string)

In [3]:
dividends = pd.DataFrame()
close = pd.DataFrame()
vol = pd.DataFrame()
for stock in ticker:
    if stock == "CASH_USD":
        continue
    stock = stock.replace('.', '-')
    df = tickers.tickers[stock].history(start="2014-01-01", interval="1d", auto_adjust = True)
    dividends[stock] = df["Dividends"]
    close[stock] = df["Close"]
    vol[stock] = df["Volume"]

C:\Users\tbastia\AppData\Local\Temp\ipykernel_28236\467724417.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dividends[stock] = df["Dividends"]
C:\Users\tbastia\AppData\Local\Temp\ipykernel_28236\467724417.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  close[stock] = df["Close"]
C:\Users\tbastia\AppData\Local\Temp\ipykernel_28236\467724417.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

# Smart Beta Portfolio

## Index Weights

In [4]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    dollar_volume = close*volume
    dollar_volume_weights = dollar_volume.div(dollar_volume.sum(axis=1), axis=0)
    return dollar_volume_weights
    
index_weights = generate_dollar_volume_weights(close, vol)

In [5]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)  

def plot_topn_holdings(weights, n):

    last_day = weights.tail(1).melt().sort_values(by=['value'], ascending=False).head(n)

    fig = go.Figure(data=[go.Bar(x=last_day['variable'], y=last_day['value'])])
    fig.update_yaxes(tickformat=".1%")
    fig.update_layout(
        title = "Top 5 Holdings"
        )

    fig.show()

In [6]:
plot_topn_holdings(index_weights, 5)

## Portfolio Weights

In [7]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    dividend_sum = dividends.cumsum()
    dividend_weights = dividend_sum.div(dividend_sum.sum(axis=1), axis=0)
    return dividend_weights

etf_weights = calculate_dividend_weights(dividends)

In [8]:
plot_topn_holdings(etf_weights, 5)

## Returns

In [9]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    returns = prices/prices.shift()-1
    return returns

returns = generate_returns(close)

## Weighted Returns

In [10]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    weighted_returns = returns*weights
    return weighted_returns

index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)

## Cumulative Returns

In [11]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    daily_return = returns.sum(axis=1)+1
    cumulative_returns = daily_return.cumprod()
    return cumulative_returns

index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)

In [15]:
def plot_cum_returns(index_return, etf_return):
    # create figure
    fig = go.Figure()
    
    # add trace
    fig.add_trace(
        go.Scatter(
            x = index_return.index,
            y = index_return,
            line = dict(color="blue", width=2),
            name = "Index"
            )
        )

    fig.add_trace(
    go.Scatter(
        x = etf_return.index,
        y = etf_return,
        line = dict(color="green", width=2),
        name = "ETF"
        )
    )

    
    # update layout
    fig.update_layout(
        title = "Smart Beta ETF vs. Index"
        )
    
    # show figure
    iplot(fig)   

In [16]:
plot_cum_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns)

## Tracking Error

In [14]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    difference = etf_returns_by_date-benchmark_returns_by_date
    tracking_error = np.sqrt(252)*np.std(difference, ddof=1)

    return tracking_error

smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

Smart Beta Tracking Error: 0.11462657443186866
